# Import Libraries:
Download the dependencies that I will need.

In [1]:
import numpy as np
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json 
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim
import requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0 --yes
import folium
import lxml.html as lh
print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.9.11          |           py36_0         147 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    ca-certificates-2019.9.11  |       hecc5488_0         144 KB  conda-forge
    openssl-1.1.1c             |       h516909a_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.49-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

# Convert html to dataframe:
Define the data I'm going to use and convert to pandas dataframe

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
url

'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [3]:
page = requests.get(url)
doc = lh.fromstring(page.content)
tr_elements = doc.xpath('//tr')
tr_elements

[<Element tr at 0x7fb8d66c3318>,
 <Element tr at 0x7fb8d66e1548>,
 <Element tr at 0x7fb8d66e14f8>,
 <Element tr at 0x7fb8d66e1598>,
 <Element tr at 0x7fb8d66e15e8>,
 <Element tr at 0x7fb8d66e1638>,
 <Element tr at 0x7fb8d66e1688>,
 <Element tr at 0x7fb8d66e1908>,
 <Element tr at 0x7fb8d66e1958>,
 <Element tr at 0x7fb8d66e19a8>,
 <Element tr at 0x7fb8d66e19f8>,
 <Element tr at 0x7fb8d66e1a48>,
 <Element tr at 0x7fb8d66e1a98>,
 <Element tr at 0x7fb8d66e1ae8>,
 <Element tr at 0x7fb8d66e1b38>,
 <Element tr at 0x7fb8d66e1b88>,
 <Element tr at 0x7fb8d66e1bd8>,
 <Element tr at 0x7fb8d66e1c28>,
 <Element tr at 0x7fb8d66e1c78>,
 <Element tr at 0x7fb8d66e1cc8>,
 <Element tr at 0x7fb8d66e1d18>,
 <Element tr at 0x7fb8d66e1d68>,
 <Element tr at 0x7fb8d66e1db8>,
 <Element tr at 0x7fb8d66e1e08>,
 <Element tr at 0x7fb8d66e1e58>,
 <Element tr at 0x7fb8d66e1ea8>,
 <Element tr at 0x7fb8d66e1ef8>,
 <Element tr at 0x7fb8d66e1f48>,
 <Element tr at 0x7fb8d66e1f98>,
 <Element tr at 0x7fb8d6256048>,
 <Element 

In [4]:
[len(T) for T in tr_elements[:12]]

[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]

In [5]:
tr_elements = doc.xpath('//tr')
#Create empty list
col=[]
i=0
#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print ('%d:"%s"'%(i,name))
    col.append((name,[]))
    print()

1:"Postcode"

2:"Borough"

3:"Neighbourhood
"



In [6]:
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 10, the //tr data is not from our table 
    if len(T)!=3:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1

In [7]:
[len(C) for (title,C) in col]

[288, 288, 288]

In [8]:
Dict={title:column for (title,column) in col}
TorontoData=pd.DataFrame(Dict)
TorontoData.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned\n
1,M2A,Not assigned,Not assigned\n
2,M3A,North York,Parkwoods\n
3,M4A,North York,Victoria Village\n
4,M5A,Downtown Toronto,Harbourfront\n


In [9]:
TorontoData.columns = TorontoData.columns.str.replace('\n', '')
TorontoData.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned\n
1,M2A,Not assigned,Not assigned\n
2,M3A,North York,Parkwoods\n
3,M4A,North York,Victoria Village\n
4,M5A,Downtown Toronto,Harbourfront\n


In [10]:
TorontoDatas = TorontoData.replace(r'[,\n"\']','', regex=True)
TorontoDatas

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queens Park,Not assigned
9,M8A,Not assigned,Not assigned


In [11]:
TorontoDatas.describe()

,Postcode,Borough,Neighbourhood
count,288,288,288
unique,180,12,209
top,M8Y,Not assigned,Not assigned
freq,8,77,78


In [34]:
TorontoDatas=TorontoDatas.rename(columns = {'Postcode':'PostalCode'})
TorontoDatas

,PostalCode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queens Park,Not assigned
9,M8A,Not assigned,Not assigned


# Clean data

a. Process cells where Borough has an assigned value

In [35]:
TorontoDatass = TorontoDatas[TorontoDatas.Borough !='Not assigned'].reset_index(drop=True)
TorontoDatass.head()

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


In [13]:
print (TorontoDatass.columns.tolist())

['Postcode', 'Borough', 'Neighbourhood']


b. Join the values in Neighbourhood where Postcode has the same value

In [36]:
TorontoDatasss = TorontoDatass.groupby(['PostalCode', 'Borough'])['Neighbourhood'].apply(', '.join).reset_index()
TorontoDatasss

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


c. Set Neighbourhood value equal to Borough value when the value in Neighbourhood is Not assigned:

In [38]:
TorontoDatass['Neighbourhood'] = np.where(TorontoDatass['Neighbourhood'] == "Not assigned", TorontoDatass['Borough'], TorontoDatass['Neighbourhood'])
TorontoDatass = TorontoDatass.sort_values(by ='PostalCode').reset_index()
TorontoDatass

,index,PostalCode,Borough,Neighbourhood
0,8,M1B,Scarborough,Rouge
1,9,M1B,Scarborough,Malvern
2,23,M1C,Scarborough,Port Union
3,22,M1C,Scarborough,Rouge Hill
4,21,M1C,Scarborough,Highland Creek
5,33,M1E,Scarborough,Guildwood
6,34,M1E,Scarborough,Morningside
7,35,M1E,Scarborough,West Hill
8,39,M1G,Scarborough,Woburn
9,43,M1H,Scarborough,Cedarbrae


d. Show number of rows for the dataframe

In [33]:
TorontoDatass.shape

(211, 4)